# Making city posters

In [ ]:
%reload_ext autoreload
%autoreload 2

from srai.regionalizers import geocode_to_region_gdf
from srai.loaders.osm_loaders.filters import BASE_OSM_GROUPS_FILTER
from srai.loaders import OSMPbfLoader

import numpy as np

import utils
import warnings
warnings.simplefilter("ignore")

# Paper size as global variable
PAPER_SIZE = utils.PAPER_SIZES['A2']

## Example: Plano Piloto (Brasília)

Specify the city

In [ ]:
CITY = "Plano Piloto"
COUNTRY = "Brazil"

area_name = f"{CITY}, {COUNTRY}"
area_name

Download the area and plot it interactively, to see if the region you want has been captured correctly.

In [ ]:
area = geocode_to_region_gdf(area_name)
area.explore(height=PAPER_SIZE[0], width=PAPER_SIZE[1])

We can filter city properties to plo using standardized feature names. You can check possible names in the `BASE_OSM_GROUPS_FILTER` dictionary in the `srai` package.

In [ ]:
print("Possible feature filter names:")
for key in BASE_OSM_GROUPS_FILTER.keys():
    print("  " + key)
    for child_key in BASE_OSM_GROUPS_FILTER[key].keys():
        print("    " + str(child_key))
        try:
            for item in BASE_OSM_GROUPS_FILTER[key][child_key]:
                print("      " + str(item))
        except TypeError:
            pass
    print()

You can access these properties from the `area` we downloaded earlier via a `OSMPbfLoader()` object. Simply pass the filter names you want to use to the latter's `.load()` method as dictionary keys, setting their values to `True` (otherwise, they are `False` by default).

In [ ]:
features = (
    OSMPbfLoader()
    .load(area, {"highway": True, "water": True, "waterway": True})
    .clip(area)
)
features.head(3)

We can see these features in an interactive map, to check if they are what we want, or if they where properly captured.

In [ ]:
features.explore(height=PAPER_SIZE[0], width=PAPER_SIZE[1])

Once the desired features are been selected, we can draw the corresponding poster using the function `utils.plot_poster`

In [ ]:
# Check the lat/lon range of the features
lon_min, lat_min, lon_max, lat_max = features.total_bounds
print(f"Lat range: {lat_min:.2f} to {lat_max:.2f}")
print(f"Lon range: {lon_min:.2f} to {lon_max:.2f}")

In [ ]:
# Pin center to get a better picture
# pin_center = None  # Set this to None to disable pinning
pin_center = (-47.870, -15.785)  # (lon, lat)

pin_center = utils.parse_pin_center(pin_center,
                                    lat_min,
                                    lat_max,
                                    lon_min,
                                    lon_max)

In [ ]:
# Set zoom level to better center the map
lat_min, lat_max, lon_min, lon_max = utils.zoom(
    lat_min,
    lat_max,
    lon_min,
    lon_max,
    pin_center,
    zoom_level=.5
)
print(f"Lat range: {lat_min:.2f} to {lat_max:.2f}")
print(f"Lon range: {lon_max:.2f} to {lon_max:.2f}")

In [ ]:
# Set figure size to display here
reduce_factor = 45
figsize = tuple(np.array(PAPER_SIZE) / reduce_factor)

# Plot the poster
fig, ax = utils.plot_poster(
    features,
    feature_props = {
        "water": {"color": "#a8e1e6"},
        "waterway": {"color": "#a8e1e6"},
        "highway": {
            "color": "#181818",
            "linewidth": 0.5,
            "markersize": 0.5,
            },
    },
    lat_lim=(lat_min, lat_max),
    lon_lim=(lon_min, lon_max),
    figsize=figsize,
    # show_axis=True,  # Use this for debugging. Set to False for final version
)

# Save the figure
fig.savefig("poster.pdf",
            bbox_inches="tight",
            pad_inches=0,
            dpi=300)

fig.show()

## Explore other cities

In [ ]:
CITY = "Warsaw"
COUNTRY = "Poland"

area_name = f"{CITY}, {COUNTRY}"
area_name

In [ ]:
area = geocode_to_region_gdf(area_name)
features = (
    OSMPbfLoader()
    .load(area, {"highway": True, "water": True, "waterway": True})
    .clip(area)
)
features.head(3)

In [ ]:
# Check the lat/lon range of the features
lon_min, lat_min, lon_max, lat_max = features.total_bounds
print(f"Lat range: {lat_min:.2f} to {lat_max:.2f}")
print(f"Lon range: {lon_min:.2f} to {lon_max:.2f}")

In [ ]:
# Pin center to get a better picture
pin_center = None  # Set this to None to disable pinning
# pin_center = (-47.870, -15.785)  # (lon, lat)

pin_center = utils.parse_pin_center(pin_center,
                                    lat_min,
                                    lat_max,
                                    lon_min,
                                    lon_max)

print("Pinned center: ({:.2f}, {:.2f})".format(*pin_center))

In [ ]:
# Set zoom level to better center the map
lat_min, lat_max, lon_min, lon_max = utils.zoom(
    lat_min,
    lat_max,
    lon_min,
    lon_max,
    pin_center,
    zoom_level=0.9
)
print(f"Lat range: {lat_min:.2f} to {lat_max:.2f}")
print(f"Lon range: {lon_max:.2f} to {lon_max:.2f}")

In [ ]:
# Set figure size to display here
reduce_factor = 45
figsize = tuple(np.array(PAPER_SIZE) / reduce_factor)

# Plot the poster
fig, ax = utils.plot_poster(
    features,
    lat_lim=(lat_min, lat_max),
    lon_lim=(lon_min, lon_max),
    figsize=figsize,
    show_axis=True,  # Use this for debugging. Set to False for final version
)

fig.show()

In [ ]:
(lat_max - lat_min) / (lon_max - lon_min)